In [ ]:
import time
import os
import argparse

import numpy as np


In [ ]:
""" White box MDPs (wbmdp) defined five-tuple M=(S,A,c,P,gamma) """

import sys

import numpy as np

import numpy.linalg as la

import sklearn
import sklearn.pipeline
import sklearn.kernel_approximation
import sklearn.linear_model

from sklearn.exceptions import ConvergenceWarning
from sklearn.utils._testing import ignore_warnings

TOL = 1e-10

# Right (0), Down (1), Left (2), Up (3)
DIRS = [(1,0), (0,1), (-1,0), (0,-1)]

class MDPModel():
    """ Base MDP class """
    def __init__(self, n_states, n_actions, c, P, gamma, rho=None, seed=None):
        assert len(c.shape) == 2, "Input cost vector c must be a 2-D vector, recieved %d dimensions" % len(c.shape)
        assert len(P.shape) == 3, "Input cost vector c must be a 3-D tensor, recieved %d dimensions" % len(P.shape)

        assert c.shape[0] == n_states, "1st dimension of c must equal n_states=%d, was instead %d" % (n_states, c.shape[0])
        assert c.shape[1] == n_actions, "2nd dimension of c must equal n_actions=%d, was instead %d" % (n_actions, c.shape[1])
        assert P.shape[0] == n_states, "1st dimension of P must equal n_states=%d, was instead %d" % (n_states, P.shape[0])
        assert P.shape[1] == n_states, "2nd dimension of P must equal n_states=%d, was instead %d" % (n_states, P.shape[1])
        assert P.shape[2] == n_actions, "3rd dimension of P must equal n_actions=%d, was instead %d" % (n_actions, P.shape[2])
        assert 0 < gamma < 1, "Input discount gamma must be (0,1), recieved %f" % gamma

        assert 1-TOL <= np.min(np.sum(P, axis=0)), \
            "P is not stochastic, recieved a sum of %.2f at (s,a)=(%d,%d)" % ( \
                np.min(np.sum(P, axis=0)), \
                np.where(1-TOL > np.sum(P, axis=0))[0][0], \
                np.where(1-TOL > np.sum(P, axis=0))[1][0], \
            )
        assert np.max(np.sum(P, axis=0)) <= 1+TOL, \
            "P is not stochastic, recieved a sum of %.2f at (s,a)=(%d,%d)" % ( \
                np.max(np.sum(P, axis=0)), \
                np.where(1+TOL < np.sum(P, axis=0))[0][0], \
                np.where(1+TOL < np.sum(P, axis=0))[1][0], \
            )

        self.n_states = n_states
        self.n_actions = n_actions
        self.c = c
        self.P = P
        self.gamma = gamma
        if rho is None:
            rho = np.ones(self.n_states, dtype=float)/self.n_states
        self.rho = rho

        # initialize a
        self.rng = np.random.default_rng(seed)
        self.s = self.rng.integers(0, self.n_states)

        # initialize rbf for solving with linear function approx
        self.init_linear = False

    def get_advantage(self, pi):
        assert pi.shape[0] == self.n_actions, "1st dimension of pi must equal n_actions=%d, was instead %d" % (self.n_actions, pi.shape[0])
        assert pi.shape[1] == self.n_states, "2nd dimension of pi must equal n_states=%d, was instead %d" % (self.n_states, pi.shape[1])

        # sum over actions (p=s' next state, s curr state, a action)
        P_pi = np.einsum('psa,as->ps', self.P, pi)
        c_pi = np.einsum('sa,as->s', self.c, pi)

        # (I-gamma*(P^pi)')V = c^pi
        V_pi = la.solve(np.eye(self.n_states) - self.gamma*P_pi.T, c_pi)
        Q_pi = self.c + self.gamma*np.einsum('psa,p->sa', self.P, V_pi)
        psi = Q_pi - np.outer(V_pi, np.ones(self.n_actions))

        return (psi, V_pi)

    def estimate_advantage_generative_slow(self, pi, N, T):
        """
        :param N: number of Monte Carlo simulations to run per state-action pair
        :param T: duration to for each Monte Carlo simulation
        """
        Q = np.zeros((self.n_states, self.n_actions), dtype=float)

        for s in range(self.n_states):
            for a in range(self.n_actions):
                costs = 0.
                for i in range(N):
                    s_t = s
                    a_t = a
                    for t in range(T):
                        Q[s,a] += self.gamma**t * self.c[s_t,a_t]
                        s_t_next = self.rng.choice(self.P.shape[0], p=self.P[:,s_t,a_t])
                        a_t = self.rng.choice(pi.shape[0], p=pi[:,s_t])
                        s_t = s_t_next

                Q[s,a] /= N

        V_pi = np.einsum('sa,as->s', Q, pi)
        psi = Q - np.outer(V_pi, np.ones(self.n_actions, dtype=float))

        return (psi, V_pi)

    def estimate_advantage_generative(self, pi, N, T):
        """
        :param N: number of Monte Carlo simulations to run per state-action pair
        :param T: duration to for each Monte Carlo simulation
        """
        # 1 x S
        pi_sum = np.cumsum(pi, axis=0)
        # S x (SA)
        P_reshape = np.reshape(self.P, newshape=(self.P.shape[0], self.P.shape[1]*self.P.shape[2]))
        P_reshape_sum = np.cumsum(P_reshape, axis=0)

        # SA
        q = np.zeros(self.n_states*self.n_actions, dtype=float)

        for i in range(N):
            s_arr = np.kron(np.arange(self.n_states), np.ones(self.n_actions, dtype=int))
            a_arr = np.kron(np.ones(self.n_states, dtype=int), np.arange(self.n_actions))
            for t in range(T):
                q += self.gamma**t * self.c[s_arr, a_arr]

                u = self.rng.uniform(size=len(q))
                z_arr = s_arr * self.n_actions + a_arr
                s_arr = np.argmax(np.outer(u, np.ones(self.n_states)) < P_reshape_sum[:,z_arr].T, axis=1)

                u = self.rng.uniform(size=len(q))
                a_arr = np.argmax(np.outer(u, np.ones(self.n_actions)) < pi_sum[:,s_arr].T, axis=1)

        q /= N
        Q = np.reshape(q, newshape=(self.n_states, self.n_actions))

        V_pi = np.einsum('sa,as->s', Q, pi)
        psi = Q - np.outer(V_pi, np.ones(self.n_actions, dtype=float))

        return (psi, V_pi)

    def estimate_advantage_online_mc(self, pi, T, threshold=0, bootstrap=False):
        """
        https://arxiv.org/pdf/2303.04386

        :param T: duration to run Monte Carlo simulation
        :param threshold: pi(a|s) < threshold means Q(s,a)=largest value, do not visit again (rec: (1-gamma)**2/|A|)
        :return visit_len_state_action: how long the Monte carlo estimate is at every state-aciton pair
        """
        costs = np.zeros(T, dtype=float)
        states = np.zeros(T, dtype=int)
        actions = np.zeros(T, dtype=int)

        for t in range(T):
            states[t] = self.s
            actions[t] = self.rng.choice(pi.shape[0], p=pi[:,states[t]])
            costs[t] = self.c[states[t], actions[t]]
            self.s = self.rng.choice(self.P.shape[0], p=self.P[:,states[t],actions[t]])

        # check bootstrap
        if bootstrap and self.init_linear:
            a_t = self.rng.choice(pi.shape[0], p=pi[:,self.s])
            costs[-1] += self.gamma * self.predict([[self.s,a_t]])

        # form advantage (dp style);
        cumulative_discounted_costs = np.zeros(T, dtype=float)
        cumulative_discounted_costs[-1] = costs[-1]
        for t in range(T-2,-1,-1):
            cumulative_discounted_costs[t] = costs[t] + self.gamma*cumulative_discounted_costs[t+1]

        Q = np.zeros((self.n_states, self.n_actions), dtype=float)
        visit_len_state_action = np.zeros((self.n_states, self.n_actions), dtype=bool)
        for t in range(T):
            (s,a) = states[t], actions[t]
            if visit_len_state_action[s,a] > 0:
                continue
            Q[s,a] = cumulative_discounted_costs[t]
            visit_len_state_action[s,a] = T-t

        # for proabibilities that are very low, set Q value to be high
        (poor_sa_a, poor_sa_s) = np.where(pi <= threshold)
        Q_max = np.max(np.abs(self.c))/(1.-self.gamma)
        Q[poor_sa_s,poor_sa_a] = Q_max

        V_pi = np.einsum('sa,as->s', Q, pi)
        psi = Q - np.outer(V_pi, np.ones(self.n_actions, dtype=float))

        return (psi, V_pi, visit_len_state_action)

    def init_estimate_advantage_online_linear(self, linear_settings):
        """
        Prepares radial basis functions for linear function approximation:

            https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.RBFSampler.html

        See also: https://github.com/dennybritz/reinforcement-learning/blob/master/FA/Q-Learning%20with%20Value%20Function%20Approximation%20Solution.ipynb

        :param X: Nxn array of inputs, where N is the number of datapoints and n is the size of the state space
	    """

        self.featurizer = sklearn.pipeline.FeatureUnion([
            # ("rbf0", RBFSampler(gamma=5.0, n_components=100)),
            ("rbf1", sklearn.kernel_approximation.RBFSampler(gamma=1.0, n_components=100)),
            # ("rbf2", RBFSampler(gamma=0.1, n_components=100)),
        ])

        X = np.vstack((
            np.kron(np.arange(self.n_states), np.ones(self.n_actions)),
            np.kron(np.ones(self.n_states), np.arange(self.n_actions)),
        )).T

        self.featurizer.fit(X)
        self.model = sklearn.linear_model.SGDRegressor(
            learning_rate=linear_settings["linear_learning_rate"],
            eta0=linear_settings["linear_eta0"],
            max_iter=linear_settings["linear_max_iter"],
            alpha=linear_settings["linear_alpha"],
            warm_start=True,
            tol=0.0,
            n_iter_no_change=linear_settings["linear_max_iter"],
            fit_intercept=True,
        )

        # We need to call partial_fit once to initialize the model or we get a
        # NotFittedError when trying to make a prediction This is quite hacky.
        self.model.partial_fit(self.featurize([X[0]]), [0])
        self.init_linear = True

    def featurize(self, X):
        return self.featurizer.transform(X).astype('float64')

    def predict(self, x):
        features = self.featurize(x)
        output = np.squeeze(self.model.predict(features))
        return output

    def get_all_sa_pairs_for_finite(self):
        X_all_sa = np.vstack((
            np.kron(np.arange(self.n_states), np.ones(self.n_actions)),
            np.kron(np.ones(self.n_states), np.arange(self.n_actions)),
        )).T
        return X_all_sa

    def custom_SGD(solver, X, y, minibatch=32):
        n_epochs = solver.max_iter
        n_consec_regress_epochs = 0
        max_regress = solver.n_iter_no_change
        frac_validation = solver.validation_fraction
        tol = solver.tol
        early_stopping = solver.early_stopping

        train_losses = []
        test_losses = []

        for i in range(n_epochs):
            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=i, shuffle=True, test_size=frac_validation)
            num_batches = int(np.ceil(len(X_train)/ minibatch))
            for j in range(num_batches):
                k_s = minibatch*j
                k_e = min(len(X_train), minibatch*(j+1))
                # mini-batch update
                solver.partial_fit(X_train[k_s:k_e], y_train[k_s:k_e])

            y_train_pred = solver.predict(X_train)
            y_test_pred = solver.predict(X_test)

            train_losses.append(la.norm(y_train_pred - y_train)**2/len(y_train))
            test_losses.append(la.norm(y_test_pred - y_test)**2/len(y_test))

            if early_stopping and len(test_losses) > 1 and test_losses[-1] > np.min(test_losses)-tol:
                n_consec_regress_epochs += 1
            else:
                n_consec_regress_epochs = 0
            if n_consec_regress_epochs == max_regress:
                print("Early stopping (stagnate)")
                break
            if train_losses[-1] <= tol:
                print("Early stopping (train loss small)")
                break

        return np.array(train_losses), np.array(test_losses)

    # https://scikit-learn.org/stable/auto_examples/linear_model/plot_sgd_early_stopping.html#sphx-glr-auto-examples-linear-model-plot-sgd-early-stopping-py
    @ignore_warnings(category=ConvergenceWarning)
    def estimate_advantage_online_linear(self, pi, T):
        """
        Use Monte Carlo simulation to obtain partial Q function.  We use linear
        function approximation with bootstrap to update sampled sa pairs and
        fill in missing sa pairs.

        :param T: duration to run Monte Carlo simulation
        """
        assert self.init_linear, "Run `init_estimate_advantage_online_linear` before estimating"

        # use monte carlo estimate to estimate truncated psi (threshold=0
        # ensures non-visited sa have zero value, i.e., Q[s,a]=0)
        output = self.estimate_advantage_online_mc(pi, T, threshold=0, bootstrap=True)
        (psi, V_pi, visit_len_state_action) = output
        Q = psi + np.outer(V_pi, np.ones(self.n_actions, dtype=float))

        # bootstrap remaining cost-to-go values
        # X_all_sa = self.get_all_sa_pairs_for_finite()
        # y = self.predict(X_all_sa)

        visited_sa_s, visited_sa_a = np.where(visit_len_state_action >= 1)
        X_visited_sa = np.vstack((visited_sa_s, visited_sa_a)).T
        # state-action pair index in 1D
        visited_idxs = self.n_actions * visited_sa_s + visited_sa_a

        # y = Q.flatten() + np.multiply(np.power(self.gamma, visit_len_state_action.flatten()), y)
        # visited_idxs = np.where(visit_len_state_action.flatten() > 0)[0]
        y = Q.flatten()[visited_idxs]
        # for i, (s,a) in zip(visited_idxs, X_visited_sa):
        #     y[i] = Q[s,a] + self.gamma**visit_len_state_action[s,a]*y[i]

        # training update
        # features = self.featurize(X_visited_sa)
        # self.model.fit(features, y[visited_idxs])
        # features = self.featurize(X_all_sa)
        # self.model.fit(features, y)
        features = self.featurize(X_visited_sa)
        self.model.fit(features, y)

        # predict psi_pi
        X_all_sa = self.get_all_sa_pairs_for_finite()
        q_pred = self.predict(X_all_sa)
        Q_pred = np.reshape(q_pred, newshape=(self.n_states, self.n_actions))
        V_pred = np.einsum('sa,as->s', Q_pred, pi)
        psi_pred = Q_pred - np.outer(V_pred, np.ones(self.n_actions, dtype=float))

        return (psi_pred, V_pred)

    def get_steadystate(self, pi):
        P_pi = np.einsum('psa,as->ps', self.P, pi)

        dim = P_pi.shape[0]
        Q = (P_pi.T-np.eye(dim))
        ones = np.ones(dim)
        Q = np.c_[Q,ones]
        QTQ = np.dot(Q, Q.T)

        # check singular
        try:
            if la.matrix_rank(QTQ) < QTQ.shape[0]:
                print("Singular matrix when computing stationary distribution, return zero vector")
                return np.zeros(QTQ.shape[0], dtype=float)
        except:
            # error with matrix rank
            return np.zeros(QTQ.shape[0], dtype=float)

        bQT = np.ones(dim)
        return np.linalg.solve(QTQ,bQT)

class GridWorldWithTraps(MDPModel):

    def __init__(self, length, n_traps, gamma, n_origins=-1, eps=0.05, seed=None, ergodic=False):
        """ Creates 2D gridworld with side length @length grid world with traps.

        Each step incurs a cost of +1
        @n_traps traps are randomly placed. Stepping on it will incur a high an addition cost of +5
        Reaching the target state will incur a cost of +0 and the agent will remain there.

        If :ergodic:=True mode, then reaching the target incurs a -length cost
        and the next state is a random non-target non-trap state. This ensures
        all state-action spaces can be visited after reaching the target.

        The agent can move in one of the four cardinal directions, if feasible.
        There is a @eps probability another random direction will be selected.
        """

        self.length = length
        n_states = length*length
        n_actions = 4
        n_traps = min(n_traps, n_states-1)
        if n_origins == -1:
            n_origins = n_states-n_traps-1

        # have the same set of traps, origins, and traps
        rng = np.random.default_rng(seed)
        rnd_pts = rng.choice(length*length, replace=False, size=n_traps+1+n_origins)

        rng = np.random.default_rng(seed)
        self.traps = traps = rnd_pts[:n_traps]
        self.origins = origins = rnd_pts[n_traps:n_traps+n_origins]
        rho = np.zeros(length*length, dtype=float)
        rho[origins] = 1./len(origins)
        self.target = target = rnd_pts[-1]
        if len(origins) < 10:
            print("  Origins at ", np.sort(origins))

        P = np.zeros((n_states, n_states, n_actions), dtype=float)
        c = np.zeros((n_states, n_actions), dtype=float)

        def fill_gw_P_at_xy(P, x, y):
            """
            Applies standard probability in the 4 cardinal directions provided by @x and @y

            :param x: x-axis locations of source we want to move from
            :param y: y-axis locations of source we want to move from
            :param length: length of x and y-axis
            :param eps: random probability of moving in another direction
            """
            s = length*y+x
            for a in range(4):
                next_x = np.clip(x + DIRS[a][0], 0, length-1)
                next_y = np.clip(y + DIRS[a][1], 0, length-1)
                next_s = length*next_y+next_x
                P[next_s, s, a] = (1.-eps)

                # random action
                for b in range(4):
                    if b==a: continue
                    next_x = np.clip(x + DIRS[b][0], 0, length-1)
                    next_y = np.clip(y + DIRS[b][1], 0, length-1)
                    next_s = length*next_y+next_x
                    P[next_s, s, a] += eps/3 # add to not over-write

        # handle corners
        for i in range(4):
            x = (length-1)*(i%2)
            y = (length-1)*(i//2)
            fill_gw_P_at_xy(P, x, y)

        # vertical edges
        for i in range(2):
            x = (length-1)*i
            y = np.arange(1,length-1)
            fill_gw_P_at_xy(P, x, y)

        # horizontal edges
        for i in range(2):
            y = (length-1)*i
            x = np.arange(1,length-1)
            fill_gw_P_at_xy(P, x, y)

        # inner squares
        x = np.kron(np.ones(length, dtype=int), np.arange(1, length-1))
        y = np.kron(np.arange(1, length-1), np.ones(length, dtype=int))
        fill_gw_P_at_xy(P, x, y)

        # target
        if ergodic:
            # rnd_pts = rng.choice(length*length, replace=False, size=n_traps+1)
            # non_target_nor_trap = np.setdiff1d(np.arange(length*length), rnd_pts)

            P[:,target,:] = 0
            # go to random non-target non-trap location
            P[origins,target,:] = 1./len(origins)
        else:
            P[:,target,:] = 0
            # stay at target
            P[target,target,:] = 1.

        # apply trap cost
        c[:,:] = 1.
        c[traps,:] = 10.
        c[target,:] = -10.

        super().__init__(n_states, n_actions, c, P, gamma, rho, seed)

    def get_target(self):
        return self.target

    def init_agent(self):
        self.agent = self.rng.choice(self.origins)
        self.curr_time = 0
        return self.agent

    def step(self, action):
        self.agent = self.rng.choice(self.P.shape[0], p=self.P[:,self.agent, action])
        self.curr_time += 1

        if self.agent == self.target:
            print("Target reached in %d steps! Resetting" % self.curr_time)
            self.curr_time = 0
            self.agent = self.rng.choice(self.origins)
        elif self.agent in self.traps:
            print("Target hit a trap")
        elif self.curr_time >= 50:
            print("Agent stalled, resetting")
            self.curr_time = 0
            self.agent = self.rng.choice(self.origins)

        return self.agent

    def print_grid(self):
        # next_s = length*next_y+next_x
        if not hasattr(self, "grid_pt"):
            self.grid_pt = [ [' ']*self.length for _ in range(self.length) ]
            # target
            self.grid_pt[self.target//self.length][self.target % self.length] = 'D'
            for trap in self.traps:
                (y,x) = (trap // self.length, trap % self.length)
                self.grid_pt[y][x] = 'T'

        # agent
        self.grid_pt[self.agent//self.length][self.agent% self.length] = '*'

        msg = "|" + "-"*(self.length*2-1) + "|\n"
        for row in self.grid_pt:
            msg += "|" + ':'.join(row) + "|\n"
        msg += "|" + "-"*(self.length*2-1) + "|\n"
        print(msg, end="")

        # agent
        if self.agent not in self.traps:
            self.grid_pt[self.agent//self.length][self.agent% self.length] = ' '
        else:
            self.grid_pt[self.agent//self.length][self.agent% self.length] = 'T'




# Bigger gridworld, traps increased proportionally

### length = 30, traps = 113

In [ ]:

def get_env(name, gamma, seed=None):

    if name == "gridworld":
        env = GridWorldWithTraps(30, int(np.ceil(50*(1.5*1.5))), gamma, seed=seed, ergodic=True)
    else:
        raise Exception("Unknown env_name=%s" % name)

    return env

In [ ]:
""" Basic stochastic PMD """


def policy_update(pi, psi, eta):
    """ Closed-form solution with PMD subproblem

    :param pi (np.ndarray): current policy
    :param psi (np.ndarray): current policy's advantage function
    :param eta (float): step size
    :return (np.ndarray): next policy (should be same shape as @pi)
    """

    # Apply psi to update the policy
    updated = pi * np.exp(-eta * psi.T)

    # Normalize across actions
    updated /= np.sum(updated, axis=0, keepdims=True)

    # different from solns but i think normalisation probably takes care of it? (same outputs)

    return updated

def simulate_agent(env, pi, T=100):
    s = env.init_agent()
    env.print_grid()
    rng = np.random.default_rng()

    for _ in range(T):
        a = rng.choice(pi.shape[0], p=pi[:,s])
        s = env.step(a)
        time.sleep(0.5)
        #env.print_grid()

def train(settings):
    env = get_env(settings['env_name'], settings['gamma'], settings['seed'])

    # print formatter
    exp_metadata = ["Iter", "Est f(pi)", "Est f(pi*)", "Est gap"]
    row_format ="{:>5}|{:>10}|{:>10}|{:>10}"
    print("")
    print(row_format.format(*exp_metadata))
    print("-" * (35+len(exp_metadata)-1))

    # initial policy
    pi_t = np.ones((env.n_actions, env.n_states), dtype=float)/env.n_actions

    agg_psi_t = np.zeros((env.n_states, env.n_actions), dtype=float)
    agg_V_t = np.zeros(env.n_states, dtype=float)

    s_time = time.time()
    for t in range(settings["n_iters"]):
        (psi_t, V_t) = env.estimate_advantage_generative(pi_t, settings["N"], settings["T"])
        adv_gap = np.max(-agg_psi_t, axis=1)/(1.-env.gamma)

        alpha_t = 1./(t+1)
        agg_psi_t = (1.-alpha_t)*agg_psi_t + alpha_t*psi_t
        agg_V_t = (1.-alpha_t)*agg_V_t + alpha_t*V_t

        if ((t+1) <= 100 and (t+1) % 5 == 0) or (t+1) % 25==0:
            print(row_format.format(
                t+1,
                "%.2e" % np.dot(env.rho, V_t),
                "%.2e" % np.dot(env.rho, agg_V_t - adv_gap),
                "%.2e" % (np.dot(env.rho, V_t) - np.dot(env.rho, agg_V_t - adv_gap)),
            ))

        # eta_t = settings["alpha"]/(t+1)**0.5
        eta_t = settings["alpha"]/(settings["n_iters"])**0.5
        pi_t = policy_update(pi_t, psi_t, eta_t)

    print("Total runtime: %.2fs" % (time.time() - s_time))

    (true_psi_t, true_V_t) = env.get_advantage(pi_t)
    adv_gap = np.max(-true_psi_t, axis=1)/(1.-env.gamma)

    print("=== Final performance metric ===")
    print("  f(pi_k):   %.4e\n  f(pi*) lb: %.4e\n  Gap:       %.4e" % (
        np.dot(env.rho, true_V_t),
        np.dot(env.rho, true_V_t - adv_gap),
        np.dot(env.rho, adv_gap),
    ))
    print("="*40)

    if settings["visual"] and settings['env_name'] == 'gridworld':
        simulate_agent(env, pi_t)

In [ ]:
alphas = [0.01, 0.1, 0.5, 5, 10, 100]
for alpha in alphas:
  print("======================Current alpha================================")
  print(alpha)
  print("====================================================================")
  settings = dict({
        "alpha": alpha,
        "visual": "store_true",
        "N": 1,
        "T": 50,
        "gamma": 0.9,
        "env_name": 'gridworld',
        "n_iters": 200,
        "seed": 0, # fixed seed so that i get the same env each time
        "advantage": "generative"
    })
  train(settings)

======================Current alpha================================
0.01

 Iter| Est f(pi)|Est f(pi*)|   Est gap
--------------------------------------
    5|  1.92e+01| -2.30e+01|  4.22e+01
   10|  1.91e+01| -1.48e+01|  3.39e+01
   15|  1.87e+01| -1.19e+01|  3.06e+01
   20|  1.85e+01| -1.02e+01|  2.87e+01
   25|  1.84e+01| -9.18e+00|  2.76e+01
   30|  1.80e+01| -8.51e+00|  2.66e+01
   35|  1.77e+01| -8.16e+00|  2.59e+01
   40|  1.74e+01| -7.63e+00|  2.50e+01
   45|  1.74e+01| -7.11e+00|  2.45e+01
   50|  1.70e+01| -6.78e+00|  2.38e+01
   55|  1.68e+01| -6.43e+00|  2.32e+01
   60|  1.66e+01| -6.17e+00|  2.28e+01
   65|  1.65e+01| -5.89e+00|  2.24e+01
   70|  1.63e+01| -5.59e+00|  2.19e+01
   75|  1.62e+01| -5.34e+00|  2.15e+01
   80|  1.58e+01| -5.10e+00|  2.09e+01
   85|  1.57e+01| -4.86e+00|  2.05e+01
   90|  1.56e+01| -4.70e+00|  2.03e+01
   95|  1.54e+01| -4.51e+00|  1.99e+01
  100|  1.53e+01| -4.32e+00|  1.96e+01
  125|  1.44e+01| -3.42e+00|  1.78e+01
  150|  1.38e+01| -2.64e+00| 

In [ ]:
alphas = [1]
for alpha in alphas:
  print("======================Current alpha================================")
  print(alpha)
  print("====================================================================")
  settings = dict({
        "alpha": alpha,
        "visual": "store_true",
        "N": 1,
        "T": 50,
        "gamma": 0.9,
        "env_name": 'gridworld',
        "n_iters": 200,
        "seed": 0, # fixed seed so that i get the same env each time
        "advantage": "generative"
    })
  train(settings)

======================Current alpha================================
1

 Iter| Est f(pi)|Est f(pi*)|   Est gap
--------------------------------------
    5|  1.10e+01| -1.21e+01|  2.31e+01
   10|  9.91e+00| -1.27e+00|  1.12e+01
   15|  9.61e+00|  2.28e+00|  7.33e+00
   20|  9.52e+00|  3.86e+00|  5.66e+00
   25|  9.37e+00|  4.82e+00|  4.55e+00
   30|  9.27e+00|  5.42e+00|  3.85e+00
   35|  9.05e+00|  5.82e+00|  3.24e+00
   40|  9.01e+00|  6.11e+00|  2.89e+00
   45|  8.88e+00|  6.34e+00|  2.54e+00
   50|  8.85e+00|  6.51e+00|  2.34e+00
   55|  8.82e+00|  6.65e+00|  2.17e+00
   60|  8.68e+00|  6.77e+00|  1.91e+00
   65|  8.67e+00|  6.87e+00|  1.81e+00
   70|  8.63e+00|  6.93e+00|  1.70e+00
   75|  8.70e+00|  7.01e+00|  1.69e+00
   80|  8.59e+00|  7.07e+00|  1.52e+00
   85|  8.54e+00|  7.13e+00|  1.42e+00
   90|  8.57e+00|  7.18e+00|  1.39e+00
   95|  8.53e+00|  7.22e+00|  1.31e+00
  100|  8.46e+00|  7.26e+00|  1.20e+00
  125|  8.38e+00|  7.42e+00|  9.63e-01
  150|  8.31e+00|  7.52e+00|  7.

$\alpha^* = 5$

In [ ]:
Ts = [25,100]
for T in Ts:
  print("======================Current T================================")
  print(T)
  print("====================================================================")
  settings = dict({
        "alpha": 5,
        "visual": "store_true",
        "N": 1,
        "T": T,
        "gamma": 0.9,
        "env_name": 'gridworld',
        "n_iters": 200,
        "seed": 0, # fixed seed so that i get the same env each time
        "advantage": "generative"
    })
  train(settings)

======================Current T================================
25

 Iter| Est f(pi)|Est f(pi*)|   Est gap
--------------------------------------
    5|  8.99e+00| -7.12e+00|  1.61e+01
   10|  8.60e+00|  1.09e+00|  7.51e+00
   15|  8.24e+00|  3.37e+00|  4.87e+00
   20|  8.23e+00|  4.42e+00|  3.80e+00
   25|  8.07e+00|  5.03e+00|  3.04e+00
   30|  7.98e+00|  5.42e+00|  2.56e+00
   35|  7.97e+00|  5.71e+00|  2.27e+00
   40|  7.87e+00|  5.91e+00|  1.96e+00
   45|  7.89e+00|  6.07e+00|  1.82e+00
   50|  7.92e+00|  6.20e+00|  1.72e+00
   55|  7.76e+00|  6.31e+00|  1.45e+00
   60|  7.83e+00|  6.41e+00|  1.42e+00
   65|  7.80e+00|  6.49e+00|  1.32e+00
   70|  7.74e+00|  6.56e+00|  1.18e+00
   75|  7.87e+00|  6.62e+00|  1.25e+00
   80|  7.74e+00|  6.68e+00|  1.06e+00
   85|  7.89e+00|  6.73e+00|  1.16e+00
   90|  7.70e+00|  6.77e+00|  9.32e-01
   95|  7.68e+00|  6.80e+00|  8.78e-01
  100|  7.69e+00|  6.84e+00|  8.54e-01
  125|  7.75e+00|  6.97e+00|  7.78e-01
  150|  7.63e+00|  7.06e+00|  5.67e

### length = 40, traps = 200



In [ ]:
def get_env(name, gamma, seed=None):

    if name == "gridworld":
        env = GridWorldWithTraps(40, 200, gamma, seed=seed, ergodic=True)
    else:
        raise Exception("Unknown env_name=%s" % name)

    return env

In [ ]:
""" Basic stochastic PMD """


def policy_update(pi, psi, eta):
    """ Closed-form solution with PMD subproblem

    :param pi (np.ndarray): current policy
    :param psi (np.ndarray): current policy's advantage function
    :param eta (float): step size
    :return (np.ndarray): next policy (should be same shape as @pi)
    """

    # Apply psi to update the policy
    updated = pi * np.exp(-eta * psi.T)

    # Normalize across actions
    updated /= np.sum(updated, axis=0, keepdims=True)

    # different from solns but i think normalisation probably takes care of it? (same outputs)

    return updated

def simulate_agent(env, pi, T=100):
    s = env.init_agent()
    env.print_grid()
    rng = np.random.default_rng()

    for _ in range(T):
        a = rng.choice(pi.shape[0], p=pi[:,s])
        s = env.step(a)
        time.sleep(0.5)
        #env.print_grid()

def train(settings):
    env = get_env(settings['env_name'], settings['gamma'], settings['seed'])

    # print formatter
    exp_metadata = ["Iter", "Est f(pi)", "Est f(pi*)", "Est gap"]
    row_format ="{:>5}|{:>10}|{:>10}|{:>10}"
    print("")
    print(row_format.format(*exp_metadata))
    print("-" * (35+len(exp_metadata)-1))

    # initial policy
    pi_t = np.ones((env.n_actions, env.n_states), dtype=float)/env.n_actions

    agg_psi_t = np.zeros((env.n_states, env.n_actions), dtype=float)
    agg_V_t = np.zeros(env.n_states, dtype=float)

    s_time = time.time()
    for t in range(settings["n_iters"]):
        (psi_t, V_t) = env.estimate_advantage_generative(pi_t, settings["N"], settings["T"])
        adv_gap = np.max(-agg_psi_t, axis=1)/(1.-env.gamma)

        alpha_t = 1./(t+1)
        agg_psi_t = (1.-alpha_t)*agg_psi_t + alpha_t*psi_t
        agg_V_t = (1.-alpha_t)*agg_V_t + alpha_t*V_t

        if ((t+1) <= 100 and (t+1) % 5 == 0) or (t+1) % 25==0:
            print(row_format.format(
                t+1,
                "%.2e" % np.dot(env.rho, V_t),
                "%.2e" % np.dot(env.rho, agg_V_t - adv_gap),
                "%.2e" % (np.dot(env.rho, V_t) - np.dot(env.rho, agg_V_t - adv_gap)),
            ))

        # eta_t = settings["alpha"]/(t+1)**0.5
        eta_t = settings["alpha"]/(settings["n_iters"])**0.5
        pi_t = policy_update(pi_t, psi_t, eta_t)

    print("Total runtime: %.2fs" % (time.time() - s_time))

    (true_psi_t, true_V_t) = env.get_advantage(pi_t)
    adv_gap = np.max(-true_psi_t, axis=1)/(1.-env.gamma)

    print("=== Final performance metric ===")
    print("  f(pi_k):   %.4e\n  f(pi*) lb: %.4e\n  Gap:       %.4e" % (
        np.dot(env.rho, true_V_t),
        np.dot(env.rho, true_V_t - adv_gap),
        np.dot(env.rho, adv_gap),
    ))
    print("="*40)

    if settings["visual"] and settings['env_name'] == 'gridworld':
        simulate_agent(env, pi_t)

In [ ]:
alphas = [0.1, 0.5, 1, 5, 10]
for alpha in alphas:
  print("======================Current alpha================================")
  print(alpha)
  print("====================================================================")
  settings = dict({
        "alpha": alpha,
        "visual": "store_true",
        "N": 1,
        "T": 50,
        "gamma": 0.9,
        "env_name": 'gridworld',
        "n_iters": 200,
        "seed": 0, # fixed seed so that i get the same env each time
        "advantage": "generative"
    })
  train(settings)

======================Current alpha================================
0.1

 Iter| Est f(pi)|Est f(pi*)|   Est gap
--------------------------------------
    5|  1.69e+01| -2.23e+01|  3.92e+01
   10|  1.50e+01| -1.21e+01|  2.71e+01
   15|  1.36e+01| -7.73e+00|  2.13e+01
   20|  1.28e+01| -5.07e+00|  1.78e+01
   25|  1.21e+01| -3.05e+00|  1.51e+01
   30|  1.16e+01| -1.46e+00|  1.31e+01
   35|  1.12e+01| -2.62e-01|  1.14e+01
   40|  1.10e+01|  6.95e-01|  1.03e+01
   45|  1.08e+01|  1.49e+00|  9.30e+00
   50|  1.06e+01|  2.18e+00|  8.41e+00
   55|  1.05e+01|  2.78e+00|  7.73e+00
   60|  1.04e+01|  3.27e+00|  7.16e+00
   65|  1.03e+01|  3.72e+00|  6.63e+00
   70|  1.03e+01|  4.10e+00|  6.24e+00
   75|  1.03e+01|  4.44e+00|  5.83e+00
   80|  1.03e+01|  4.73e+00|  5.53e+00
   85|  1.02e+01|  4.98e+00|  5.25e+00
   90|  1.02e+01|  5.23e+00|  4.98e+00
   95|  1.01e+01|  5.44e+00|  4.71e+00
  100|  1.02e+01|  5.64e+00|  4.56e+00
  125|  1.02e+01|  6.37e+00|  3.79e+00
  150|  1.01e+01|  6.85e+00|  

In [ ]:
alphas = [5, 10]
for alpha in alphas:
  print("======================Current alpha================================")
  print(alpha)
  print("====================================================================")
  settings = dict({
        "alpha": alpha,
        "visual": "store_true",
        "N": 1,
        "T": 50,
        "gamma": 0.9,
        "env_name": 'gridworld',
        "n_iters": 200,
        "seed": 0, # fixed seed so that i get the same env each time
        "advantage": "generative"
    })
  train(settings)

======================Current alpha================================
5

 Iter| Est f(pi)|Est f(pi*)|   Est gap
--------------------------------------
    5|  1.02e+01| -6.63e+00|  1.68e+01
   10|  9.97e+00|  2.16e+00|  7.81e+00
   15|  9.85e+00|  4.64e+00|  5.21e+00
   20|  9.80e+00|  5.82e+00|  3.98e+00
   25|  9.72e+00|  6.50e+00|  3.22e+00
   30|  9.68e+00|  6.97e+00|  2.71e+00
   35|  9.57e+00|  7.28e+00|  2.29e+00
   40|  9.59e+00|  7.53e+00|  2.06e+00
   45|  9.60e+00|  7.71e+00|  1.89e+00
   50|  9.53e+00|  7.87e+00|  1.66e+00
   55|  9.51e+00|  7.99e+00|  1.52e+00
   60|  9.50e+00|  8.09e+00|  1.42e+00
   65|  9.50e+00|  8.18e+00|  1.32e+00
   70|  9.53e+00|  8.25e+00|  1.28e+00
   75|  9.44e+00|  8.31e+00|  1.14e+00
   80|  9.46e+00|  8.36e+00|  1.10e+00
   85|  9.49e+00|  8.42e+00|  1.07e+00
   90|  9.50e+00|  8.46e+00|  1.04e+00
   95|  9.41e+00|  8.50e+00|  9.07e-01
  100|  9.48e+00|  8.54e+00|  9.42e-01
  125|  9.45e+00|  8.67e+00|  7.76e-01
  150|  9.38e+00|  8.77e+00|  6.